In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, min_periods=1, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, min_periods=1, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, min_periods=1, adjust=False).mean()
    data['Impulsive MACD'] = data['MACD'] - data['Signal_Line']
    return data

In [3]:
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

In [4]:
def find_buy_sell_positions(stock_data):
    buy_positions = []
    sell_positions = []

    for i in range(1, len(stock_data)):
        if stock_data['Impulsive MACD'][i] != 0:
            if stock_data['Impulsive MACD'][i] > 0:
                buy_positions.append(i)
            elif stock_data['RSI'][i] > 65:
                sell_positions.append(i)

    return buy_positions, sell_positions

In [5]:
file_path = '^NSEI.csv'
stock_data = pd.read_csv(file_path)

In [6]:
stock_data = calculate_rsi(stock_data)
stock_data = calculate_macd(stock_data)
buy_positions, sell_positions = find_buy_sell_positions(stock_data)

In [10]:

buy_dates = stock_data['Date'].iloc[buy_positions]
sell_dates = stock_data['Date'].iloc[sell_positions]
buy_dates_df = pd.DataFrame({'Buy_Date': buy_dates})
sell_dates_df = pd.DataFrame({'Sell_Date': sell_dates})

In [12]:
buy_dates_df.to_csv('buy_dates.csv', index=False)
sell_dates_df.to_csv('sell_dates.csv', index=False)